In [117]:
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

In [118]:
datasetPath = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
sentences = WikiCorpus(datasetPath).get_texts()
print(sentences)

<generator object WikiCorpus.get_texts at 0x7ffb4f158f10>


In [119]:
nltk.download('punkt')
nltk.download('reuters')
rSentences  = reuters.sents()
print(rSentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [120]:
def probilityCalculation(model):
  for nextWord in model:
    nextWords = model[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count

In [121]:
def sigleWordProbability(model,wordCount):
  for word in model:
    model[word]/=wordCount

In [122]:
def convertToLower(para):
  if type(para)==str:
    return para.lower()
  return para

In [123]:
model4 = defaultdict(lambda: set())
model5 = defaultdict(lambda: set())

def calculateWordCount(model1,model2,model3,sentences):
  wordCount = 0
  for sentence in sentences:
    #print(sentence)
    for word in sentence:
      wordCount+=1
      model1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      nextWord = convertToLower(nextWord)
      model2[nextWord][previousWord1]+=1
      model3[nextWord][previousWord2]+=1
      model4[previousWord1].add(nextWord)
      model5[previousWord2].add(nextWord)

  return wordCount

In [124]:
model1 = defaultdict(lambda:0)
model2 = defaultdict(lambda: defaultdict(lambda:0))
model3 = defaultdict(lambda: defaultdict(lambda:0))

wikiWordCount = calculateWordCount(model1,model2,model3,wikiSentences)
print(wikiWordCount)

0


In [125]:
rWordCount = calculateWordCount(model1,model2,model3,rSentences)
print(reutersWordCount)
probilityCalculation(model2)
probilityCalculation(model3)
total_word = wikiWordCount + rWordCount
sigleWordProbability(model1,total_word)

1720917


In [126]:

probabilities = []
def WordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in model4[previousWord1] & model5[previousWord2]:
    weight = model1[nextWord]*model2[nextWord][previousWord1]*model3[nextWord][previousWord2]
    probabilities.append((nextWord,weight))

In [127]:
WordSuggestionByTrigram('my','name')
maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*maxProbabilityWords[:10])

('happening', 4.022899955707872e-07) ('really', 2.905427745789018e-07) ('viewed', 2.7345202313308405e-07) ('going', 2.2215798860066998e-07) ('important', 1.8948441820363163e-07) ('now', 1.4650845972111864e-07) ('required', 1.2676321438212316e-07) ('traditionally', 1.2083064349217088e-07) ('thought', 1.2074504917564753e-07) ('believed', 1.1188746568101528e-07)


In [129]:
while(True):
    text = input("Sentence: ")
    if text == "stop":
        break
    
    else:
        try:
            probabilities = []
            text = text.split(" ")
            WordSuggestionByTrigram(text[0],text[1])
            probabilities.sort(key=lambda o:o[1],reverse=True)
            print(*probabilities[:10])
            
        except:
            continue

Sentence: what is
('happening', 4.022899955707872e-07) ('really', 2.905427745789018e-07) ('viewed', 2.7345202313308405e-07) ('going', 2.2215798860066998e-07) ('important', 1.8948441820363163e-07) ('now', 1.4650845972111864e-07) ('required', 1.2676321438212316e-07) ('traditionally', 1.2083064349217088e-07) ('thought', 1.2074504917564753e-07) ('believed', 1.1188746568101528e-07)


KeyboardInterrupt: ignored